In [4]:
%%capture
%pip install jsonschema networkx numpy scipy pythreejs shapely
%pip install compas>=2.8 --no-deps
%pip install compas_notebook>=0.8.2 --no-deps
%pip install compas_model --no-deps
%pip install compas_dem --no-deps

# DEM of a Cross Vault

## Block Model

In [5]:
import pathlib
import compas
from compas_dem.models import BlockModel

FILE = pathlib.Path.cwd() / "data" / "crossvault.json"

model: BlockModel = compas.json_load(FILE)

## Visualisation

In [6]:
from compas_notebook import Viewer
from compas_notebook.config import Config

config = Config()
config.view.camera.position = [0, -13, 3]
config.view.camera.target = [0, 0, 3]

viewer = Viewer(config=config)
viewer.scene.add(model)
viewer.show()